In [5]:
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree
from geopy.distance import geodesic
import logging
import time

class FastMinimalCentroidClustering:
    def __init__(self, max_radius_km=5, verbose=True):
        """
        Initialize the clustering algorithm with optimized approach
        """
        self.max_radius_km = max_radius_km
        self.verbose = verbose
        self.centroids = None
        self.labels = None
        
        # Configure logging
        logging.basicConfig(
            level=logging.INFO, 
            format='%(asctime)s - %(levelname)s: %(message)s'
        )
        self.logger = logging.getLogger(__name__)
    
    def haversine_distance(self, point1, point2):
        """
        Calculate geodesic distance between two points
        """
        return geodesic(point1, point2).kilometers
    
    def find_minimal_centroids(self, points):
        """
        Optimized method to find minimal centroids
        """
        start_time = time.time()
        
        if self.verbose:
            self.logger.info(f"Starting clustering with max radius {self.max_radius_km} km")
            self.logger.info(f"Total points to cluster: {len(points)}")
        
        # Convert points to list of tuples for distance calculation
        point_tuples = [tuple(point) for point in points]
        
        # Initialize variables
        labels = np.full(len(points), -1)
        centroids = []
        
        # Track unclustered points efficiently
        unclustered_mask = np.ones(len(points), dtype=bool)
        
        # Iteration counter for centroids
        centroid_count = 0
        
        while unclustered_mask.any():
            # Find indices of unclustered points
            unclustered_indices = np.where(unclustered_mask)[0]
            
            # Find the point that covers the most unclustered points
            max_coverage = 0
            best_centroid_index = None
            
            for candidate_index in unclustered_indices:
                candidate_point = point_tuples[candidate_index]
                
                # Use more efficient distance checking
                covered_points = [
                    i for i in unclustered_indices 
                    if self.haversine_distance(candidate_point, point_tuples[i]) <= self.max_radius_km
                ]
                
                # Update best centroid if more points are covered
                if len(covered_points) > max_coverage:
                    max_coverage = len(covered_points)
                    best_centroid_index = candidate_index
            
            # If no points can be clustered, break
            if best_centroid_index is None:
                if self.verbose:
                    self.logger.warning("Unable to cluster remaining points.")
                break
            
            # Add centroid
            centroid_location = point_tuples[best_centroid_index]
            centroids.append(centroid_location)
            centroid_count += 1
            
            # Verbose logging about centroid
            if self.verbose:
                self.logger.info(f"Found Centroid {centroid_count}:")
                self.logger.info(f"  Location: {centroid_location}")
                self.logger.info(f"  Covering {max_coverage} points")
            
            # Mark points within radius as clustered
            for i in range(len(points)):
                if unclustered_mask[i] and self.haversine_distance(point_tuples[i], centroid_location) <= self.max_radius_km:
                    labels[i] = centroid_count - 1
                    unclustered_mask[i] = False
            
            # Log remaining unclustered points
            remaining_points = np.sum(unclustered_mask)
            if self.verbose:
                self.logger.info(f"Remaining unclustered points: {remaining_points}")
        
        # Final processing
        self.centroids = np.array(centroids)
        self.labels = labels
        
        # Runtime and summary
        end_time = time.time()
        runtime = end_time - start_time
        
        if self.verbose:
            self.logger.info("Clustering Complete")
            self.logger.info(f"Total Centroids: {centroid_count}")
            self.logger.info(f"Total Runtime: {runtime:.2f} seconds")
        
        return labels, self.centroids
    
    def validate_clustering(self, points):
        """
        Validate clustering with more efficient checking
        """
        point_tuples = [tuple(point) for point in points]
        
        valid_points = 0
        invalid_points = []
        
        for i, (point, label) in enumerate(zip(point_tuples, self.labels)):
            if label == -1:
                if self.verbose:
                    self.logger.error(f"Point {i} remains unclustered")
                return False
            
            centroid = self.centroids[label]
            dist = self.haversine_distance(point, centroid)
            
            if dist > self.max_radius_km:
                if self.verbose:
                    self.logger.error(f"Point {i} exceeds radius constraint")
                    self.logger.error(f"  Point: {point}")
                    self.logger.error(f"  Centroid: {centroid}")
                    self.logger.error(f"  Distance: {dist} km (Max allowed: {self.max_radius_km} km)")
                invalid_points.append(i)
            else:
                valid_points += 1
        
        if self.verbose:
            self.logger.info(f"Validation Summary:")
            self.logger.info(f"  Total Points: {len(points)}")
            self.logger.info(f"  Valid Points: {valid_points}")
            if invalid_points:
                self.logger.warning(f"  Invalid Points: {invalid_points}")
        
        return len(invalid_points) == 0

def cluster_from_csv(file_path, lat_col='latitude', lng_col='longitude', max_radius_km=5, verbose=True):
    """
    Perform minimal centroid clustering on data from a CSV file
    """
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Extract latitude and longitude
    points = df[[lat_col, lng_col]].values
    
    # Perform clustering
    clusterer = FastMinimalCentroidClustering(
        max_radius_km=max_radius_km, 
        verbose=verbose
    )
    labels, centroids = clusterer.find_minimal_centroids(points)
    
    # Validate clustering
    is_valid = clusterer.validate_clustering(points)
    
    # Add cluster labels to the dataframe
    df['cluster'] = labels
    
    return df, centroids

# Example usage (replace with your CSV file path)
def main():
    # Replace 'your_data.csv' with the path to your actual CSV file
    try:
        result_df, centroids = cluster_from_csv(
            'data-1733399738886.csv',  # Replace with your file path
            lat_col='lat',  # Replace with your latitude column name
            lng_col='lng',  # Replace with your longitude column name
            max_radius_km=5  # Adjust radius as needed
        )
        
        # Optional: save results
        result_df.to_csv('clustered_results.csv', index=False)
        
    except FileNotFoundError:
        print("CSV file not found. Please check the file path.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Uncomment to run
main()

# Demonstration of how to set up
# print("Please replace 'your_data.csv' with your actual file path and column names.")

2024-12-05 17:54:31,578 - INFO: Starting clustering with max radius 5 km
2024-12-05 17:54:31,579 - INFO: Total points to cluster: 70313


KeyboardInterrupt: 